In [1]:
import ast
import os

In [2]:
class FunctionDependencyVisitor(ast.NodeVisitor):
    def __init__(self):
        self.functions = {}
        self.current_function = None

    def visit_FunctionDef(self, node):
        self.current_function = node.name
        self.functions[node.name] = {
            'calls': [],
            'lineno': node.lineno,
            'col_offset': node.col_offset
        }
        self.generic_visit(node)
        self.current_function = None

    def visit_Call(self, node):
        if self.current_function:
            func_name = self._get_call_name(node)
            if func_name:
                self.functions[self.current_function]['calls'].append(func_name)
        self.generic_visit(node)

    def _get_call_name(self, node):
        if isinstance(node.func, ast.Name):
            return node.func.id
        elif isinstance(node.func, ast.Attribute):
            return node.func.attr
        return None

def parse_python_files(directory):
    visitor = FunctionDependencyVisitor()
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as source:
                    tree = ast.parse(source.read(), filename=file_path)
                    visitor.visit(tree)
    return visitor.functions

 

In [3]:
import networkx as nx

def build_dependency_graph(functions):
    G = nx.DiGraph()
    for func, details in functions.items():
        G.add_node(func, lineno=details['lineno'], col_offset=details['col_offset'])
        for called_func in details['calls']:
            G.add_edge(func, called_func)
    return G

In [3]:
parse_python_files("./pyrenew/")

{'predict': {'calls': ['matmul', 'transform'], 'lineno': 46, 'col_offset': 4},
 'sample': {'calls': ['tensor',
   'tensor',
   'zeros',
   'range',
   'sample',
   'Normal'],
  'lineno': 25,
  'col_offset': 4},
 '__init__': {'calls': [], 'lineno': 11, 'col_offset': 4},
 '__repr__': {'calls': [], 'lineno': 21, 'col_offset': 4},
 'spread_draws': {'calls': ['spread_draws', 'get_samples'],
  'lineno': 444,
  'col_offset': 4},
 'plot_posterior': {'calls': ['plot_posterior', 'spread_draws'],
  'lineno': 460,
  'col_offset': 4},
 'validate_discrete_dist_vector': {'calls': ['flatten',
   'all',
   'ValueError',
   'isclose',
   'sum',
   'tensor',
   'ValueError'],
  'lineno': 4,
  'col_offset': 0},
 'reverse_discrete_dist_vector': {'calls': ['flip'],
  'lineno': 69,
  'col_offset': 0},
 'new_double_convolve_scanner': {'calls': [], 'lineno': 26, 'col_offset': 0},
 '_new_scanner': {'calls': ['t1',
   'dot',
   't2',
   'dot',
   'cat',
   'unsqueeze',
   'item'],
  'lineno': 36,
  'col_offset':

In [7]:
def find_all_dependencies(G, function_name):
    dependencies = nx.descendants(G, function_name)
    return dependencies

# Example function to show dependencies
def show_dependencies(directory, function_name):
    functions = parse_python_files(directory)
    G = build_dependency_graph(functions)
    dependencies = find_all_dependencies(G, function_name)
    print(f"Dependencies for {function_name}: {dependencies}")

# Integration
show_dependencies("./test/", "test_glm_prediction")

Dependencies for test_glm_prediction: {'assert_close', 'matmul', 'predict', 'zeros', 'set_rng_seed', 'GLMPrediction', 'sample', 'expand', 'isinstance', 'Uniform', 'tensor', 'Normal', 'IdentityTransform', 'keys'}


In [6]:
!pwd

/Users/krue284/Local Files/ASKEM/multisignal-epi-inference/model/src
